# Building a Comedy Sketch Creation System with Moya

This notebook demonstrates how to create a multi-agent comedy sketch creation system using Moya for the multi-agent framework with OpenAI agents.

## Installation

First, let's install the required packages.

In [67]:
import os
os.environ["CHROMADB_CLIENT"] = "true"

## Import Required Libraries

In [68]:
import os
import json
from dotenv import load_dotenv
from openai import AzureOpenAI

In [69]:
from moya.agents.openai_agent import OpenAIAgent, OpenAIAgentConfig
from moya.agents.remote_agent import RemoteAgent, RemoteAgentConfig
from moya.classifiers.llm_classifier import LLMClassifier
from moya.orchestrators.multi_agent_orchestrator import MultiAgentOrchestrator
from moya.registry.agent_registry import AgentRegistry
from moya.tools.ephemeral_memory import EphemeralMemory
from moya.memory.in_memory_repository import InMemoryRepository
from moya.tools.tool_registry import ToolRegistry

In [70]:
# Load environment variables from .env file
load_dotenv()

# Get Azure OpenAI credentials
azure_api_key = os.environ.get("AZURE_azure_api_key")
azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
azure_api_version = os.environ.get("AZURE_OPENAI_API_VERSION")

# Define Azure deployment name - you'll need to replace this with your actual deployment name
deployment_name = "gpt-4o"  # Replace with your Azure deployment name

# Create an Azure OpenAI client
azure_client = AzureOpenAI(
    api_key=azure_api_key,
    api_version=azure_api_version,
    azure_endpoint=azure_endpoint
)

# Verify the environment variables are loaded
print("Azure OpenAI API configuration loaded from .env file")

Azure OpenAI API configuration loaded from .env file


## Create Agent Registry and Tool Registry

Setting up the foundation for our multi-agent system.

In [71]:
# Create a tool registry to manage tools that agents can use
tool_registry = ToolRegistry()

# Add EphemeralMemory tool to allow agents to remember information
EphemeralMemory.configure_memory_tools(tool_registry)

# Create an agent registry to manage our agents
agent_registry = AgentRegistry()

## Define OpenAI Agents

We'll create five comedy sketch creation agents with different roles.

In [72]:
# Set up API key - replace with your actual OpenAI API key
azure_api_key = os.getenv("azure_api_key", "your-api-key-here")


In [73]:
# Create a Concept Generator agent
concept_generator_config = OpenAIAgentConfig(
    agent_name="concept_generator",
    agent_type="openai",
    description="Proposes unique comedic premises, absurd scenarios, and humorous character ideas based on themes or genres.",
    system_prompt="""You are an expert comedy concept generator. Your specialty is creating fresh, 
    engaging comedy sketch premises that align with user preferences. You excel at developing 
    absurd scenarios, unique character concepts, and innovative comedic situations. Focus on 
    originality while ensuring the concepts have strong comedic potential.""",
    model_name="gpt-4o",  # You can change this to your preferred model
    api_key=azure_api_key,
    tool_registry=tool_registry
)
concept_generator = OpenAIAgent(concept_generator_config)
concept_generator.client = azure_client

# Create a Concept Selector agent (NEW)
concept_selector_config = OpenAIAgentConfig(
    agent_name="concept_selector",
    agent_type="openai",
    description="Evaluates and selects the most promising comedy concept from multiple options, providing rationale for the selection.",
    system_prompt="""You are an expert comedy concept selector. Your specialty is evaluating 
    multiple comedy sketch premises and selecting the most promising one based on originality, 
    comedic potential, and audience appeal. Provide a clear rationale for your selection and 
    suggest minor refinements to strengthen the chosen concept.""",
    model_name="gpt-4o",
    api_key=azure_api_key,
    tool_registry=tool_registry
)
concept_selector = OpenAIAgent(concept_selector_config)
concept_selector.client = azure_client

# Create a Dialogue Writer agent (NEW)
dialogue_writer_config = OpenAIAgentConfig(
    agent_name="dialogue_writer",
    agent_type="openai",
    description="Creates initial dialogue for comedy sketches based on selected concepts.",
    system_prompt="""You are an expert comedy dialogue writer. Your specialty is creating 
    the initial dialogue for comedy sketches based on selected concepts. Focus on natural 
    conversational flow while establishing character voices and building in comedic setups. 
    Your dialogue should establish the premise clearly and set up opportunities for humor.""",
    model_name="gpt-4o",
    api_key=azure_api_key,
    tool_registry=tool_registry
)
dialogue_writer = OpenAIAgent(dialogue_writer_config)
dialogue_writer.client = azure_client

# Create an Adaptor agent (NEW)
adaptor_config = OpenAIAgentConfig(
    agent_name="adaptor",
    agent_type="openai",
    description="Adapts comedy sketch content to suit specific platforms, audiences, or performance contexts.",
    system_prompt="""You are an expert comedy adaptor. Your specialty is taking comedy sketch 
    content and adapting it to specific platforms, audiences, or performance contexts. You 
    understand the nuances of different media and how to optimize comedy for various delivery 
    methods. Focus on maintaining the core humor while making subtle changes to tone, pacing, and references 
    to suit the target context. """,
    model_name="gpt-4o",
    api_key=azure_api_key,
    tool_registry=tool_registry
)
adaptor = OpenAIAgent(adaptor_config)
adaptor.client = azure_client

# Create a Dialogue Refiner agent (keep similar to existing)
dialogue_refiner_config = OpenAIAgentConfig(
    agent_name="dialogue_refiner",
    agent_type="openai",
    description="Polishes and enhances existing dialogue with witty one-liners, callbacks, and improved comedic timing.",
    system_prompt="""You are a skilled comedy dialogue refiner who excels at polishing existing dialogue. 
    Your expertise is in enhancing timing, adding witty one-liners, creating effective callbacks, 
    and improving the overall flow of comedic conversations. Ensure proper setup and delivery 
    for maximum comedic effect while maintaining the established character voices. YOU MUST INCREASE THE NUMBER OF LAUGHS IN THE DIALOGUE.""",
    model_name="gpt-4o",
    api_key=azure_api_key,
    tool_registry=tool_registry
)
dialogue_refiner = OpenAIAgent(dialogue_refiner_config)
dialogue_refiner.client = azure_client

# Create a Structure & Timing agent (keep similar to existing)
structure_timing_config = OpenAIAgentConfig(
    agent_name="structure_timing",
    agent_type="openai",
    description="Organizes comedic escalation, beats, and timing cues to create well-paced sketches.",
    system_prompt="""You are an expert in comedy sketch structure and timing. You excel at organizing 
    comedic escalation, beats, and timing cues to create well-paced sketches. Your role is to ensure 
    that sketches flow naturally and land effectively, avoiding pacing issues and maximizing comedic impact. 
    Provide clear guidance on sketch structure, including intros, escalation points, and satisfying endings.""",
    model_name="gpt-4o-mini",
    api_key=azure_api_key,
    tool_registry=tool_registry
)
structure_timing = OpenAIAgent(structure_timing_config)
structure_timing.client = azure_client

# Register all agents with the agent registry
agent_registry.register_agent(concept_generator)
agent_registry.register_agent(concept_selector)  # New
agent_registry.register_agent(dialogue_writer)   # New
agent_registry.register_agent(adaptor)           # New
agent_registry.register_agent(dialogue_refiner)
agent_registry.register_agent(structure_timing)

## Create a Classifier

We need a classifier to route messages to the appropriate agent.

In [74]:
# Reference:
# class LLMClassifier(BaseClassifier):
#     """LLM-based classifier for agent selection."""

#     def __init__(self, llm_agent: Agent, default_agent: str):
#         """
#         Initialize with an LLM agent for classification.
        
#         :param llm_agent: An agent that will be used for classification
#         :param default_agent: The default agent to use if no specialized match is found
#         """
#         self.llm_agent = llm_agent
#         self.default_agent = default_agent

# Create a classifier agent for routing messages
classifier_agent_config = OpenAIAgentConfig(
    agent_name="classifier",
    agent_type="openai",
    description="Routes messages to the appropriate specialized agent",
    system_prompt="You are a classifier that determines which specialized agent should handle a given request.",
    model_name="gpt-4o-mini",
    api_key=azure_api_key,
    tool_registry=tool_registry
)
classifier_agent = OpenAIAgent(classifier_agent_config)
classifier_agent.client = azure_client

# Create an LLM-based classifier to route messages to appropriate agents
classifier = LLMClassifier(
    llm_agent=classifier_agent,
    default_agent="concept_generator"  # Using concept_generator as default agent
)

## Set up Multi-Agent Orchestrator

Now we'll create a multi-agent orchestrator to manage the interactions between agents.

In [75]:
# Create a multi-agent orchestrator to manage interactions
orchestrator = MultiAgentOrchestrator(
    agent_registry=agent_registry,
    classifier=classifier
)

## Define Multi-Agent Workflow

Let's create a workflow for our comedy sketch creation project.

In [76]:
# Get conversation summary from memory
thread_id = "comedy_sketch_001"
summary_tool = tool_registry.get_tool("get_summary")
if summary_tool:
    summary = summary_tool.function(thread_id=thread_id)
    print(f"Conversation summary:\n{summary}\n")


Conversation summary:




# Pipeline


In [77]:
# This cell takes user input for a comedy sketch and stores the final script

import os
from IPython.display import Markdown, display
from datetime import datetime

# Create output and logs directories if they don't exist
os.makedirs("output", exist_ok=True)
os.makedirs("logs", exist_ok=True)


In [78]:
def create_sketch(topic, audience="YouTube", format_type="video"):
    """
    Create a comedy sketch using the multi-agent system in specified order
    
    Parameters:
    topic (str): The main topic or theme for the comedy sketch
    audience (str): Target audience platform (e.g., YouTube, TikTok, live theater)
    format_type (str): Format of the sketch (e.g., video, audio, text)
    
    Returns:
    str: The final comedy sketch script
    """
    # Create a new thread ID for this request
    thread_id = f"sketch_request_{topic.replace(' ', '_').lower()}"
    
    # Initialize logging - Create a dedicated folder for this sketch
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    log_folder = f"logs/{topic.replace(' ', '_').lower()}_{timestamp}"
    os.makedirs(log_folder, exist_ok=True)
    
    def log_to_file(step_number, step_name, message_type, content):
        """Helper function to log each step to a separate file"""
        filename = f"{log_folder}/step_{step_number:02d}_{step_name}.md"
        with open(filename, "w") as log_file:
            log_file.write(f"# Step {step_number}: {step_name}\n\n")
            log_file.write(f"## {message_type}\n\n")
            log_file.write(f"{content}\n\n")
    
    # Start logging with a metadata file
    with open(f"{log_folder}/metadata.md", "w") as meta_file:
        meta_file.write(f"# Comedy Sketch Creation: {topic}\n\n")
        meta_file.write(f"- **Date:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        meta_file.write(f"- **Topic:** {topic}\n")
        meta_file.write(f"- **Audience:** {audience}\n")
        meta_file.write(f"- **Format:** {format_type}\n\n")
        meta_file.write("## Process Steps\n\n")
        meta_file.write("1. Concept Generation\n")
        meta_file.write("2. Concept Selection\n")
        meta_file.write("3. Initial Dialogue Writing\n")
        meta_file.write("4. Platform/Audience Adaptation\n")
        meta_file.write("5. Dialogue Refinement\n")
        meta_file.write("6. Structure & Timing\n")
    
    # Format the user request
    user_request = f"I need a comedy sketch about {topic}, suitable for a {audience} audience in {format_type} format."
    
    # Log the initial request
    log_to_file(0, "initial_request", "User Input", user_request)
    
    # Store user request in memory
    EphemeralMemory.store_message(thread_id=thread_id, sender="user", content=user_request)
    print(f"Creating a comedy sketch about {topic} for {audience}...\n")
    
    # Step 1: Concept Generation
    concept_task = f"""Generate 3 unique comedy sketch concepts about {topic}. 
    Each concept should include a brief premise, key characters, and the central comedic tension. 
    Add some absurd traits to some of the characters.
    Make the concepts fresh, engaging, and suitable for a {audience} audience."""
    
    log_to_file(1, "concept_generation", "Prompt to Concept Generator", concept_task)
    
    print("Step 1: Generating concepts...")
    concept_response = orchestrator.orchestrate(
        thread_id=thread_id,
        user_message=concept_task,
        agent_name="concept_generator"
    )
    
    # Store the concept generator's response
    EphemeralMemory.store_message(thread_id=thread_id, sender="concept_generator", content=concept_response)
    log_to_file(1, "concept_generation", "Response from Concept Generator", concept_response)
    
    # Get only the available information from memory 
    previous_output = concept_response
    
    # Step 2: Concept Selection
    selection_task = f"""Review the comedy sketch concepts and select the one with a natural setting and absurd characters. Provide a brief explanation for your choice, highlighting what makes this concept 
    stand out in terms of suitability for a {audience} audience."""
    
    print("Step 2: Selecting the best concept...")
    # Pass only the previous output to this step
    full_selection_prompt = selection_task + "\n\nConcepts to review:\n" + previous_output
    log_to_file(2, "concept_selection", "Prompt to Concept Selector", full_selection_prompt)
    
    selection_response = orchestrator.orchestrate(
        thread_id=thread_id,
        user_message=full_selection_prompt,
        agent_name="concept_selector"
    )
    
    # Store the concept selector's response
    EphemeralMemory.store_message(thread_id=thread_id, sender="concept_selector", content=selection_response)
    log_to_file(2, "concept_selection", "Response from Concept Selector", selection_response)
    
    # Update previous output for the next step
    previous_output = selection_response
    
    # Step 3: Initial Dialogue Writing
    dialogue_task = f"""Based on the selected concept, write the initial dialogue for this comedy sketch about {topic}.
    Create natural-sounding, but extremely absurd conversations that establish character voices and set up comedic situations.
    The characters must show conviction to what they are saying and dialogues must align with their personalities."""
    
    print("Step 3: Writing initial dialogue...")
    # Pass only the selected concept to this step
    full_dialogue_prompt = dialogue_task + "\n\nSelected concept:\n" + previous_output
    log_to_file(3, "dialogue_writing", "Prompt to Dialogue Writer", full_dialogue_prompt)
    
    dialogue_response = orchestrator.orchestrate(
        thread_id=thread_id,
        user_message=full_dialogue_prompt,
        agent_name="dialogue_writer"
    )
    
    # Store the dialogue writer's response
    EphemeralMemory.store_message(thread_id=thread_id, sender="dialogue_writer", content=dialogue_response)
    log_to_file(3, "dialogue_writing", "Response from Dialogue Writer", dialogue_response)
    
    # Update previous output for the next step
    previous_output = dialogue_response
    
    # Step 4: Adaptation for Platform/Audience
    adaptation_task = f"""Adapt the sketch dialogue to specifically suit a {audience} audience in {format_type} format.
    Adjust tone, references, and comedic elements as needed while preserving the core humor.
    Consider platform-specific features and audience preferences in your adaptation."""
    
    print("Step 4: Adapting for platform and audience...")
    # Pass only the dialogue to this step
    full_adaptation_prompt = adaptation_task + "\n\nContent to adapt:\n" + previous_output
    log_to_file(4, "platform_adaptation", "Prompt to Adaptor", full_adaptation_prompt)
    
    adaptation_response = orchestrator.orchestrate(
        thread_id=thread_id,
        user_message=full_adaptation_prompt,
        agent_name="adaptor"
    )
    
    # Store the adaptor's response
    EphemeralMemory.store_message(thread_id=thread_id, sender="adaptor", content=adaptation_response)
    log_to_file(4, "platform_adaptation", "Response from Adaptor", adaptation_response)
    
    # Update previous output for the next step
    previous_output = adaptation_response
    
    # Step 5: Dialogue Refinement
    refinement_task = f"""Review and refine the dialogue for the comedy sketch about {topic}.
Begin your response by listing the specific improvements you made. Add a lot of sharp and witty lines. 
"""
    
    print("Step 5: Refining dialogue...")
    # Pass only the adapted content to this step
    full_refinement_prompt = refinement_task + "\n\nDialogue to refine:\n" + previous_output
    log_to_file(5, "dialogue_refinement", "Prompt to Dialogue Refiner", full_refinement_prompt)
    
    refinement_response = orchestrator.orchestrate(
        thread_id=thread_id,
        user_message=full_refinement_prompt,
        agent_name="dialogue_refiner"
    )
    
    # Store the dialogue refiner's response
    EphemeralMemory.store_message(thread_id=thread_id, sender="dialogue_refiner", content=refinement_response)
    log_to_file(5, "dialogue_refinement", "Response from Dialogue Refiner", refinement_response)
    
    # Update previous output for the next step
    previous_output = refinement_response
    
    # Step 6: Structure & Timing
    structure_task = f"""Create a structured sketch outline with proper comedic escalation, beats, and timing cues.
    Organize the sketch so it flows naturally, building to a satisfying comedic climax.
    Include specific guidance on pacing, scene transitions, and staging for maximum comedic impact."""
    
    print("Step 6: Creating structure and timing...")
    # Pass only the refined dialogue to this step
    full_structure_prompt = structure_task + "\n\nContent to structure:\n" + previous_output
    log_to_file(6, "structure_timing", "Prompt to Structure & Timing Agent", full_structure_prompt)
    
    structure_response = orchestrator.orchestrate(
        thread_id=thread_id,
        user_message=full_structure_prompt,
        agent_name="structure_timing"
    )
    
    # Store the structure & timing agent's response
    EphemeralMemory.store_message(thread_id=thread_id, sender="structure_timing", content=structure_response)
    log_to_file(6, "structure_timing", "Response from Structure & Timing Agent", structure_response)
    
    # Format the final script
    sketch_title = f"Comedy Sketch: {topic.title()}"
    formatted_script = f"""# {sketch_title}

## Overview
A comedy sketch about {topic} created for {audience} in {format_type} format.

## Selected Concept
{selection_response}

## Script
{refinement_response}

## Structure & Timing
{structure_response}

## Platform Adaptation Notes
{adaptation_response}
"""
    
    # Save the final script to a file
    filename = f"output/{topic.replace(' ', '_').lower()}_sketch.md"
    with open(filename, "w") as f:
        f.write(formatted_script)
    
    # Log the final output
    with open(f"{log_folder}/final_output.md", "w") as final_file:
        final_file.write("# Final Comedy Sketch Output\n\n")
        final_file.write(formatted_script)
    
    print(f"\nSketch completed! Saved to {filename}")
    print(f"Complete logs saved to {log_folder}")
    
    # Display a preview
    display(Markdown(f"## Preview of '{sketch_title}'\n\n```\n{refinement_response[:500]}...\n```\n\n[Full script saved to {filename}]\n[Complete logs saved to {log_folder}]"))
    
    return formatted_script

In [79]:
# Run this cell to create your custom comedy sketch!

# Replace these values with your desired parameters
topic = "Dental clinic"  # The main topic/theme for the comedy sketch
audience = "YouTube"  # Target platform (YouTube, TikTok, Netflix, live theater, etc.)
format_type = "2 minute video"  # Format (video, audio, live performance, etc.)

# Create the sketch
final_script = create_sketch(
    topic=topic,
    audience=audience, 
    format_type=format_type
)

Creating a comedy sketch about Dental clinic for YouTube...

Step 1: Generating concepts...
Step 2: Selecting the best concept...
Step 3: Writing initial dialogue...
Step 4: Adapting for platform and audience...
Step 5: Refining dialogue...
Step 6: Creating structure and timing...

Sketch completed! Saved to output/dental_clinic_sketch.md
Complete logs saved to logs/dental_clinic_2025-03-16_15-45-40


## Preview of 'Comedy Sketch: Dental Clinic'

```
[dialogue_refiner] ### Improvements Made:
1. Enhanced Nurse Flossie's character by giving her a more eccentric personality and quirky one-liners.
2. Added more sharp and witty lines for Dr. Gumshoe to emphasize his hyper-dramatic nature.
3. Improved the comedic timing and flow by tightening up the dialogue and adding more punchlines.
4. Introduced a callback joke for a better comedic effect.
5. Increased the absurdity of the "dental detective" theme to heighten the comedy.

### Refined Dialogue:...
```

[Full script saved to output/dental_clinic_sketch.md]
[Complete logs saved to logs/dental_clinic_2025-03-16_15-45-40]